<h1>Deep learning project 8DM20 CSMIA</h1>

<h4>Group members:</h4>
O. Akdag - 0842508 <br>
T.P.A. Beishuizen - 0791613 <br>
A.S.A. Eskelinen - 1224333 <br>
J.H.A. Migchielsen - 0495058 <br>
L. van den Wildenberg - 0844697 <br>

In [2]:
# Import all used packages (unused packages are commented out so far)
import os
from PIL import Image as PIL_image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
from sklearn.feature_extraction import image as sklearn_image
#import matplotlib
#matplotlib.use('Agg')
#matplotlib inline
import theano
import lasagne
import time
import random
import math
import cPickle
random.seed(0)

C:\Users\s119104\AppData\Local\Continuum\Anaconda2\lib\site-packages\theano\tensor\signal\downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


<h4>Preprocessing</h4>

Before every thing can be done, at first the data images have to be read and be made in useable data types.

In [3]:
# The function that loads the data
def loadData(data_set = 'test', image = '1st_manual'):
    
    # Check for the correct input
    if data_set != 'test' and data_set != 'training':
        raise Exception('Not the right data_set file')
    if image != '1st_manual' and image != '2nd_manual' and image != 'images' and image != 'mask':
        raise Exception('Not the right image file')
    if data_set == 'training' and image == '2nd_manual':
        raise Exception('File not available')
    
    # Project and image path
    project_path = os.getcwd()
    images_path = project_path +  '/8DM20_image_dataset/' + data_set + '/' + image + '/'
    
    images = []
    
    #Open image for image (20 in total for each of them)
    for i in range(1, 21):
        
        # Find correct image number
        image_nr = str(i)
        if data_set == 'training':
            image_nr = str(20 + i)
        elif len(image_nr) == 1:
            image_nr = '0' + image_nr
            
        # Specify path for this image
        if image == '1st_manual':
            image_path = images_path + image_nr + '_manual1.gif'
        elif image == '2nd_manual':
            image_path = images_path + image_nr + '_manual2.gif'
        elif image == 'images':
            image_path = images_path + image_nr + '_' + data_set + '.tif'
        elif image == 'mask':
            image_path = images_path + image_nr + '_' + data_set + '_mask.gif'
        
        # Open and append the image to the image list
        images.append(PIL_image.open(image_path))
        
    return images

#The function that converts the channels in the images from RGB to gray
#and makes matrices from the images
def convertImageToMatrix(images):
    
    image_matrices = []
    
    for image in images:
        image_matrix = np.asarray(image.convert('RGB'))
        green_image_matrix = image_matrix[:,:,1]
        image_matrices.append(green_image_matrix)
        
    return image_matrices

All images are loaded and immediately made into matrices for further computation

In [4]:
# All test image lists
test_manual1_images = loadData('test', '1st_manual')
test_manual2_images = loadData('test', '2nd_manual')
test_raw_images = loadData('test', 'images')
test_mask_images = loadData('test', 'mask')

# Making matrices of the test images to work with
test_manual1_matrices = convertImageToMatrix(test_manual1_images)
test_manual2_matrices = convertImageToMatrix(test_manual2_images)
test_raw_matrices = convertImageToMatrix(test_raw_images)
test_mask_matrices = convertImageToMatrix(test_mask_images)

# All training image lists
training_manual1_images = loadData('training', '1st_manual')
training_raw_images = loadData('training', 'images')
training_mask_images = loadData('training', 'mask')

# Making matrices of the training images to work with
training_manual1_matrices = convertImageToMatrix(training_manual1_images)
training_raw_matrices = convertImageToMatrix(training_raw_images)
training_mask_matrices = convertImageToMatrix(training_mask_images)

The matrices are then used for further preprocessing to retrieve the suitable data for machine learning

In [26]:
kernel_size = 32 
halfsize = kernel_size/ 2

#Add side pads to the image to convert them better
images = np.pad(training_raw_matrices,((0,0),(halfsize,halfsize),(halfsize,halfsize)),'constant', constant_values=0)
masks = np.pad(training_mask_matrices,((0,0),(halfsize,halfsize),(halfsize,halfsize)),'constant', constant_values=0)
segmentations = np.pad(training_manual1_matrices,((0,0),(halfsize,halfsize),(halfsize,halfsize)),'constant', constant_values=0)

print(images.shape)
print(masks.shape)
print(segmentations.shape)

positivesamples = np.nonzero(segmentations*masks)
negativesamples = np.nonzero(masks-segmentations)  

(20L, 616L, 597L)
(20L, 616L, 597L)
(20L, 616L, 597L)


<h4>Algorithm setup</h4>
Build the LeNet network

In [27]:
def buildLeNet(X1):
    inputlayer = lasagne.layers.InputLayer(shape=(None, 1, 32, 32),input_var=X1)    
    print inputlayer.output_shape
    
    layer1 = lasagne.layers.Conv2DLayer(inputlayer, num_filters=6, filter_size=(5,5), nonlinearity=lasagne.nonlinearities.rectify, W=lasagne.init.GlorotUniform())
    print layer1.output_shape 
    
    layer2 = lasagne.layers.MaxPool2DLayer(layer1, pool_size=(2, 2))
    print layer2.output_shape 
    
    layer3 = lasagne.layers.Conv2DLayer(layer2, num_filters=16, filter_size=(5,5), nonlinearity=lasagne.nonlinearities.rectify, W=lasagne.init.GlorotUniform())
    print layer3.output_shape 
    
    layer4 = lasagne.layers.MaxPool2DLayer(layer3, pool_size=(2, 2))
    print layer4.output_shape 
    
    layer4 = lasagne.layers.flatten(layer4)
    print layer4.output_shape 
    
    layer5 = lasagne.layers.DenseLayer(layer4,num_units=120,nonlinearity=lasagne.nonlinearities.rectify)    
    print layer5.output_shape 
    
    layer6 = lasagne.layers.DenseLayer(layer5,num_units=84,nonlinearity=lasagne.nonlinearities.rectify)
    print layer6.output_shape 
    
    outputlayer = lasagne.layers.DenseLayer(layer6,num_units=2,nonlinearity=lasagne.nonlinearities.softmax)     
    print outputlayer.output_shape 
    
    return layer1, layer2, layer3, layer4, layer5, layer6, outputlayer

In [36]:
X = theano.tensor.tensor4()
Y = theano.tensor.matrix()
layer1, layer2, layer3, layer4, layer5, layer6, outputlayer = buildLeNet(X)

outputtrain = lasagne.layers.get_output(outputlayer) 
trainloss = lasagne.objectives.categorical_crossentropy(outputtrain, Y).mean() 
params = lasagne.layers.get_all_params(outputlayer, trainable=True) 
updates = lasagne.updates.adam(trainloss, params, learning_rate=0.001) 
train = theano.function(inputs=[X, Y], outputs=trainloss, updates=updates, allow_input_downcast=True)

validate = theano.function(inputs=[X, Y], outputs=trainloss, allow_input_downcast=True)

outputtest = lasagne.layers.get_output(outputlayer, deterministic=True) 
test = theano.function(inputs=[X], outputs=outputtest, allow_input_downcast=True) 

(None, 1, 32, 32)
(None, 6, 28, 28)
(None, 6, 14, 14)
(None, 16, 10, 10)
(None, 16, 5, 5)
(None, 400)
(None, 120)
(None, 84)
(None, 2)


Defining functions for training, validation and testing purposes for the previously made LeNet network

In [29]:
outputtrain = lasagne.layers.get_output(outputlayer) #function that gets the output from the network defined before.
trainloss = lasagne.objectives.categorical_crossentropy(outputtrain, Y).mean() #function that computes the mean crossentropy between the output and the real labels.
params = lasagne.layers.get_all_params(outputlayer, trainable=True) #function that gets all the parameters (weights) in the network.
updates = lasagne.updates.momentum(trainloss, params, learning_rate=0.001) #function that performs an update of the weights based on the loss.
train = theano.function(inputs=[X, Y], outputs=trainloss, updates=updates, allow_input_downcast=True) #function that does all the above based on training samples X and real labels Y.

validate = theano.function(inputs=[X, Y], outputs=trainloss, allow_input_downcast=True) #function that computes the loss without performing an update

outputtest = lasagne.layers.get_output(outputlayer, deterministic=True) #function that gets the output from the network defined before.
test = theano.function(inputs=[X], outputs=outputtest, allow_input_downcast=True) #function that gets the output based on input X

In [33]:
def make2Dpatches(samples, batch, images, patchsize, label):
    
    halfsize = int(patchsize/2)
    
    X = np.empty([len(batch),1,patchsize,patchsize],dtype=np.float32)
        
    Y = np.zeros((len(batch),2),dtype=np.int16) 
        
    for i in xrange(len(batch)):
        
        patch = images[samples[0][batch[i]],(samples[1][batch[i]]-halfsize):(samples[1][batch[i]]+halfsize),(samples[2][batch[i]]-halfsize):(samples[2][batch[i]]+halfsize)]
       
        X[i,0] = patch        
        Y[i,label] = 1 
        
    return X, Y

def save_weights(filename,network):
    with open(filename, 'wb') as f:
        cPickle.dump(lasagne.layers.get_all_param_values(network), f)

In [35]:
minibatches = 5000
minibatchsize = 200

losslist = []
        
for i in xrange(minibatches):
    posbatch = random.sample(range(len(positivesamples[0])),minibatchsize/2)
    negbatch = random.sample(range(len(negativesamples[0])),minibatchsize/2)
             
    Xpos, Ypos = make2Dpatches(positivesamples,posbatch,images,32,1)
    Xneg, Yneg = make2Dpatches(negativesamples,negbatch,images,32,0)
          
    Xtrain = np.vstack((Xpos,Xneg))
    Ytrain = np.vstack((Ypos,Yneg))
           
    loss = train(Xtrain,Ytrain)
    losslist.append(loss)
    print 'Batch: {}'.format(i)
    print 'Loss: {}'.format(loss)
                
        
plt.close('all')
plt.figure()
plt.plot(losslist)    
        
save_weights(os.getcwd() + r'\8DM20_image_dataset\trainednetwork.pkl',outputlayer)

Batch: 0
Loss: 0.692342547195
Batch: 1
Loss: 0.688156105357
Batch: 2
Loss: 0.69797347691
Batch: 3
Loss: 0.689364572218
Batch: 4
Loss: 0.689118058584
Batch: 5
Loss: 0.696670614887
Batch: 6
Loss: 0.693205139685
Batch: 7
Loss: 0.694001540232
Batch: 8
Loss: 0.695360323589
Batch: 9
Loss: 0.688431648914


<h4> Testing algorithm</h4>
Test the algorithm with putting in an image_patch and checking if the result is correct.